# To Do

1. Need to compare to classical solution

2. Something in the X calculation (exp in general) seems off. The values seem too small. There might be errors on the original code too? **FIXED** See below: it turns out we can get an energy that is too low.

# Ansatz Comparisons for the Transverse Ising Model

This notebook is supplementary to the Ising Model notebook.

The goal here is two showcase two different ansatz for the TIM, and discuss why one may work better than the other. We will compare it against the classical solution (which can be found in the notebook titled "Classical Ising Solution"). 

## Initial Ansatz

Recall from our Ising Notebook, for the two site Ising Model, we used the following ansatz:

$$ | \psi (\theta_1, \theta_2)\rangle = U(\theta_1)|0\rangle \otimes U(\theta_2)|0\rangle $$

For our first ansatz, we will simply extend this ansatz to four sites. That is:

$$ | \psi(\theta_1, \theta_2, \theta_3, \theta_4) \rangle = \bigotimes_{i=1}^4 U(\theta_i)| 0 \rangle $$

We will now see how this works. Note that we are considering **periodic boundary conditions**, meaning that the fourth site can interact with the first site. We can think of this 4 site Hamiltonian as a square with a spin on each corner. 

Since this example is slightly more complicated, rather than programming our own optimizer, we will use an optimize function from a package called Scipy. 

In [2]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
import scipy.optimize as optimize # not sure if this is working well honestly...
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector
#from qiskit.extensions import Initialize
#from qiskit_textbook.tools import random_state, array_to_latex # This import isn't working for me

import qiskit
print(qiskit.__qiskit_version__) # I add this so we know what versions are being used

{'qiskit-terra': '0.14.2', 'qiskit-aer': '0.5.2', 'qiskit-ignis': '0.3.3', 'qiskit-ibmq-provider': '0.7.2', 'qiskit-aqua': '0.7.3', 'qiskit': '0.19.6'}


In [36]:
def ansatz(params, qc): 
    # params should be a numpy array [theta1, theta2, theta3, theta4, phi]
    # This is applied in expectation, and it is assumed to be applied on a |0000> system
    # Sets theta = param, and the next two to 0. Last argument is the qubit number
    
    qc.u3(params[0], 0, 0, 0) 
    qc.u3(params[1], 0, 0, 1)
    qc.u3(params[2], 0, 0, 2) 
    qc.u3(params[3], 0, 0, 3) 
    #print(qc.draw())
    return qc

def entangled_ansatz(params, qc):
    # params should be six angles
    qc.u3(params[0], params[5],0,0)
    qc.cx(0,1)
    qc.cx(0,2)
    qc.cx(0,3)
    qc.barrier() 
    qc.u3(params[1], 0, 0, 0) 
    qc.u3(params[2], 0, 0, 1)
    qc.u3(params[3], 0, 0, 2) 
    qc.u3(params[4], 0, 0, 3)
    #print(qc.draw())
    return qc

def expZZ(index, qcO, shots_number = 1024):
    # for a given index, we measure index , index+1
    # if PBC = False, doesn't count 4, 1 
    qc = qcO.copy()
    

    qc.measure(index, 0)
    qc.measure((index + 1) % 4, 1) # mod 4 to allow for periodic boundary conditions
        
    
    backend = BasicAer.get_backend('qasm_simulator')
    counts = execute(qc, backend, shots=shots_number).result().get_counts() # executs the job
    # if 11, 00, return 1
    # if 01, 10, return -1    
    freq_pos = 0.0
    freq_neg = 0.0
    
    for key in counts.keys():
        if (key == '00') or (key == '11'):
            #print(key)
            #print('+')
            freq_pos = freq_pos + counts[key]
        else: #01, 10
            #print(key)
            #print('-')
            freq_neg = freq_neg + counts[key]
    #normalize
    freq_pos = freq_pos / shots_number
    freq_neg = freq_neg / shots_number
    #print(freq_pos)
    #print(freq_neg)
    #print(freq_pos + freq_neg)
    #print(counts)
    expZZ = (freq_pos - freq_neg)
    return expZZ



def expX(index, qcO, shots_number = 1024): 
    # for a given index, we measure index
    
    qc = qcO.copy()
    qc.h(index)
    qc.measure(index, 0) # we always measure to the first, so our results are 00 and 01
    
    backend = BasicAer.get_backend('qasm_simulator')
    counts = execute(qc, backend, shots=shots_number).result().get_counts() # executes the job
    freq_pos = 0.0
    freq_neg = 0.0
    print(counts)
    for key in counts.keys():
        if key == '00': #the result is printed in reverse order
            #print(key)
            freq_pos = freq_pos + counts[key]
            
        else:
            freq_neg = freq_neg + counts[key]
            
    freq_pos = freq_pos / shots_number
    freq_neg = freq_neg / shots_number
    expX = freq_pos - freq_neg
    print("expX is " + str(expX))
    #print("======")
    #print(plot_histogram(counts))
    
    return expX



def expectation(params, h, ansatz_type = "naive", shots_number = 1024, PBC = True): 
    # first, we run the quantum circuit
    # Then, using the resulting measurement, we use the numbers to calculate the expectation value
    # This function is hard-coded for a specific H. (I may change this later)
    # Since H = sigma^z, let's assume that we are in the z-basis. Then, we need only find the amplitudes of being 
    # in |0> and |1>. 
    # <H> = (Counts(0) - Counts(1)) / Normalization
    
    # Creating the circuit
    q = QuantumRegister(4)
    c = ClassicalRegister(2) # we only need two classical bits since we at most take 2 measurements
    qc = QuantumCircuit(q, c)
    
    if ansatz_type == "naive":
        ansatz(params, qc) # Applying the ansatz preparation on a given |0>
    else:
        entangled_ansatz(params, qc)
    
    if PBC == True:
        max_index_ZZ = 4
    else:
        max_index_ZZ = 3
    
    expX_tot = 0.0
    expZZ_tot = 0.0
    i = 0
    while i < max_index_ZZ: #iterates 01, 12, 23, 30 for the ZZ term. 01, 12, 23 if non PBC
        expX_tot = expX_tot + expX(i, qc)
        expZZ_tot = expZZ_tot + expZZ(i, qc)
        i += 1
    X_contribution = -h * expX_tot
    ZZ_contribution = -1*expZZ_tot
    print("The X contribution is: " + str(X_contribution))
    print("The ZZ contribution is: " + str(ZZ_contribution))
    exp =  X_contribution + ZZ_contribution
    return exp


def Grid_Search(initial_guess, h, step, ansatz_type):
    #minimize one value at a time
    min_theta = 0.0
    max_theta = 2*m.pi
    
    thetas = np.arange(min_theta, max_theta, step)
    #print(thetas)
    params = initial_guess
    results = np.zeros(np.size(thetas))
    
    angle_index = 0
    while (angle_index < np.size(params)):
        i = 0
        #print("Current angle_index index is: " + str(angle_index))
        
        while (i<np.size(thetas)):
            params[angle_index] = thetas[i]
            results[i] = expectation(params, h, ansatz_type)
            i+=1
        #print(results)   
        min_value = np.amin(results)
        min_index = np.where(results== min_value)[0][0] # returns the index of the first instance of the minimum value
        opt_theta = thetas[min_index]
        params[angle_index] = opt_theta
        #print(params)
        #print("Opt theta = " + str(opt_theta) + " for index: " + str(angle_index))
        #print("")
        angle_index += 1
    min_energy = expectation(params, h, ansatz_type)
    print("The minimum energy is: " + str(min_energy))
    return(params)

# h_array = np.array([0.0, 1.0, 10.0, 100.0, 1000.0])
"""
h_array = np.arange(0., 3.0, 0.5)
for h in h_array:
    print(h)
    print(Grid_Search(np.array([0., 0., 0., 0.]), h, m.pi/10, "naive" ))
    print(Grid_Search(np.array([0., 0., 0., 0., 0., 0.]), h, m.pi/10, "entangled" ))
    print()
"""
print("done")

done


In [41]:
# Grid_Search(np.array([0., 0., 0., 0.]), 1.5, m.pi/10, "naive" )
# [0.62831853, 0.9424778 , 0.62831853, 0.9424778 ]

expectation([0.62831853, 0.9424778 , 0.62831853, 0.9424778 ], 1.5, "naive")
t1=0.628
t2=0.942
t3=0.628
t4=0.942
# -1*(m.cos(t1)*m.cos(t2)*m.cos(t3)*m.cos(t4) + m.sin(t1)*m.sin(t2)*m.sin(t3)*m.sin(t4)) + (m.cos(t1)*m.sin(t2)*m.cos(t3)*m.sin(t4) + m.sin(t1)*m.cos(t2)*m.sin(t3)*m.cos(t4))
# 0.09539727536975351
q = QuantumRegister(4)
c = ClassicalRegister(4) # we only need two classical bits since we at most take 2 measurements
qc = QuantumCircuit(q, c)
ansatz([0.62831853, 0.9424778 , 0.62831853, 0.9424778 ], qc)
qc.measure(0,0)
qc.measure(1,1)
qc.measure(2,2)
qc.measure(3,3)

backend = BasicAer.get_backend('qasm_simulator') # the device to run on
result = execute(qc, backend, shots=1000).result()
counts  = result.get_counts(qc)
#print(counts)
#print(m.cos(t1)*m.cos(t2)*m.cos(t3)*m.cos(t4))
#print(m.sin(t1)*m.sin(t2)*m.sin(t3)*m.sin(t4))
#print(m.cos(t1)*m.sin(t2)*m.cos(t3)*m.sin(t4))
#print(m.sin(t1)*m.cos(t2)*m.sin(t3)*m.cos(t4))
#print(-4*(0.527) + 4*0.041)
#plot_histogram(counts)

angle = m.pi/2
expectation([angle, angle, angle, angle], 1.5, "naive")
## This makes it look like noise can let us get below the minimum, which is very weird.



{'00': 822, '01': 202}
expX is 0.60546875
{'00': 920, '01': 104}
expX is 0.796875
{'00': 807, '01': 217}
expX is 0.576171875
{'00': 929, '01': 95}
expX is 0.814453125
The X contribution is: -4.189453125
The ZZ contribution is: -1.880859375
{'00': 1024}
expX is 1.0
{'00': 1024}
expX is 1.0
{'00': 1024}
expX is 1.0
{'00': 1024}
expX is 1.0
The X contribution is: -6.0
The ZZ contribution is: -0.095703125


-6.095703125

# Qs:

1. Is it possible to get below $ h \times 4$ (granted we have noise)?
2. What is the best way to minimize?

In [26]:
def expectation(params, h = 0.0, shots_number = 1024): 
    # Creating the circuit
    q = QuantumRegister(4)
    c = ClassicalRegister(2) # we only need two classical bits since we at most take 2 measurements
    qc = QuantumCircuit(q, c)
    
    ansatz(params, qc) # Applying the ansatz preparation on a given |0>
    
    expX_tot = 0.0
    expZZ_tot = 0.0
    i = 0
    while i < 4: #iterates 01, 12, 23, 30 for the ZZ term
        expX_tot = expX_tot + expX(i, qc)
        expZZ_tot = expZZ_tot + expZZ(i, qc)
        i += 1
    X_contribution = -h * expX_tot
    ZZ_contribution = -1*expZZ_tot
    print("The X contribution is: " + str(X_contribution))
    print("The ZZ contribution is: " + str(ZZ_contribution))
    exp =  X_contribution + ZZ_contribution
    return exp

guess = np.array([m.pi/10, m.pi/10, m.pi/10, m.pi/10])
result = optimize.minimize(expectation, guess)
if result.success:
    fitted_params = result.x
    print(fitted_params)
    print(expectation(fitted_params))
else:
    raise ValueError(result.message)
    
"""


def f(x):
    a, b = x
    return a**2 + b**2 - 5



guess = [1,1]
result = optimize.minimize(f, guess)
if result.success:
    fitted_params = result.x
    print(fitted_params)
    print(f(fitted_params))
else:
    raise ValueError(result.message)
    
"""

        ┌───────────────┐
q279_0: ┤ U3(pi/10,0,0) ├
        ├───────────────┤
q279_1: ┤ U3(pi/10,0,0) ├
        ├───────────────┤
q279_2: ┤ U3(pi/10,0,0) ├
        ├───────────────┤
q279_3: ┤ U3(pi/10,0,0) ├
        └───────────────┘
c279_0: ═════════════════
                         
c279_1: ═════════════════
                         
The X contribution is: -0.0
The ZZ contribution is: -3.61328125
        ┌───────────────┐
q280_0: ┤ U3(pi/10,0,0) ├
        ├───────────────┤
q280_1: ┤ U3(pi/10,0,0) ├
        ├───────────────┤
q280_2: ┤ U3(pi/10,0,0) ├
        ├───────────────┤
q280_3: ┤ U3(pi/10,0,0) ├
        └───────────────┘
c280_0: ═════════════════
                         
c280_1: ═════════════════
                         
The X contribution is: -0.0
The ZZ contribution is: -3.626953125
        ┌───────────────┐
q281_0: ┤ U3(pi/10,0,0) ├
        ├───────────────┤
q281_1: ┤ U3(pi/10,0,0) ├
        ├───────────────┤
q281_2: ┤ U3(pi/10,0,0) ├
        ├───────────────┤
q281_3: ┤ U3(

The X contribution is: -0.0
The ZZ contribution is: -3.630859375
        ┌─────────────────┐
q300_0: ┤ U3(0.31804,0,0) ├
        ├─────────────────┤
q300_1: ┤ U3(0.31139,0,0) ├
        ├─────────────────┤
q300_2: ┤ U3(0.31749,0,0) ├
        ├─────────────────┤
q300_3: ┤ U3(0.31139,0,0) ├
        └─────────────────┘
c300_0: ═══════════════════
                           
c300_1: ═══════════════════
                           
The X contribution is: -0.0
The ZZ contribution is: -3.623046875
        ┌─────────────────┐
q301_0: ┤ U3(0.31804,0,0) ├
        ├─────────────────┤
q301_1: ┤ U3(0.31139,0,0) ├
        ├─────────────────┤
q301_2: ┤ U3(0.31749,0,0) ├
        ├─────────────────┤
q301_3: ┤ U3(0.31139,0,0) ├
        └─────────────────┘
c301_0: ═══════════════════
                           
c301_1: ═══════════════════
                           
The X contribution is: -0.0
The ZZ contribution is: -3.587890625
        ┌─────────────────┐
q302_0: ┤ U3(0.31804,0,0) ├
        ├────────────

The X contribution is: -0.0
The ZZ contribution is: -3.640625
        ┌─────────────────┐
q320_0: ┤ U3(0.31419,0,0) ├
        ├─────────────────┤
q320_1: ┤ U3(0.31414,0,0) ├
        ├─────────────────┤
q320_2: ┤ U3(0.31418,0,0) ├
        ├─────────────────┤
q320_3: ┤ U3(0.31414,0,0) ├
        └─────────────────┘
c320_0: ═══════════════════
                           
c320_1: ═══════════════════
                           
The X contribution is: -0.0
The ZZ contribution is: -3.62890625
        ┌─────────────────┐
q321_0: ┤ U3(0.31419,0,0) ├
        ├─────────────────┤
q321_1: ┤ U3(0.31414,0,0) ├
        ├─────────────────┤
q321_2: ┤ U3(0.31418,0,0) ├
        ├─────────────────┤
q321_3: ┤ U3(0.31414,0,0) ├
        └─────────────────┘
c321_0: ═══════════════════
                           
c321_1: ═══════════════════
                           
The X contribution is: -0.0
The ZZ contribution is: -3.6640625
        ┌─────────────────┐
q322_0: ┤ U3(0.31419,0,0) ├
        ├─────────────────┤

The X contribution is: -0.0
The ZZ contribution is: -3.607421875
        ┌─────────────────┐
q340_0: ┤ U3(0.31417,0,0) ├
        ├─────────────────┤
q340_1: ┤ U3(0.31415,0,0) ├
        ├─────────────────┤
q340_2: ┤ U3(0.31417,0,0) ├
        ├─────────────────┤
q340_3: ┤ U3(0.31415,0,0) ├
        └─────────────────┘
c340_0: ═══════════════════
                           
c340_1: ═══════════════════
                           
The X contribution is: -0.0
The ZZ contribution is: -3.623046875
        ┌─────────────────┐
q341_0: ┤ U3(0.31417,0,0) ├
        ├─────────────────┤
q341_1: ┤ U3(0.31415,0,0) ├
        ├─────────────────┤
q341_2: ┤ U3(0.31417,0,0) ├
        ├─────────────────┤
q341_3: ┤ U3(0.31415,0,0) ├
        └─────────────────┘
c341_0: ═══════════════════
                           
c341_1: ═══════════════════
                           
The X contribution is: -0.0
The ZZ contribution is: -3.583984375
        ┌─────────────────┐
q342_0: ┤ U3(0.31417,0,0) ├
        ├────────────

The X contribution is: -0.0
The ZZ contribution is: -3.59375
        ┌─────────────────┐
q360_0: ┤ U3(0.31417,0,0) ├
        ├─────────────────┤
q360_1: ┤ U3(0.31415,0,0) ├
        ├─────────────────┤
q360_2: ┤ U3(0.31417,0,0) ├
        ├─────────────────┤
q360_3: ┤ U3(0.31415,0,0) ├
        └─────────────────┘
c360_0: ═══════════════════
                           
c360_1: ═══════════════════
                           
The X contribution is: -0.0
The ZZ contribution is: -3.65234375
        ┌─────────────────┐
q361_0: ┤ U3(0.31417,0,0) ├
        ├─────────────────┤
q361_1: ┤ U3(0.31415,0,0) ├
        ├─────────────────┤
q361_2: ┤ U3(0.31417,0,0) ├
        ├─────────────────┤
q361_3: ┤ U3(0.31415,0,0) ├
        └─────────────────┘
c361_0: ═══════════════════
                           
c361_1: ═══════════════════
                           
The X contribution is: -0.0
The ZZ contribution is: -3.583984375
        ┌─────────────────┐
q362_0: ┤ U3(0.31417,0,0) ├
        ├─────────────────

ValueError: Desired error not necessarily achieved due to precision loss.

In [51]:
## Working on my own optimizer

def f(params):
    a,b = params
    return a + (b-m.pi)**2

def Grid_Search(initial_guess, h, step):
    #minimize one value at a time
    min_theta = 0.0
    max_theta = 2*m.pi
    
    thetas = np.arange(min_theta, max_theta, step)
    print(thetas)
    params = initial_guess
    results = np.zeros(np.size(thetas))
    
    angle_index = 0
    while (angle_index < np.size(params)):
        i = 0
        print("Current angle_index index is: " + str(angle_index))
        
        while (i<np.size(thetas)):
            params[angle_index] = thetas[i]
            results[i] = expectation(params, h)
            i+=1
            
            
        print(results)   
        min_value = np.amin(results)
        min_index = np.where(results== min_value)[0][0] # returns the index of the first instance of the minimum value
        opt_theta = thetas[min_index]
        params[angle_index] = opt_theta
        print(params)
        print("Opt theta = " + str(opt_theta) + " for index: " + str(angle_index))
        print("")
        angle_index += 1
    return(params)

Grid_Search(np.array([0., 0., 0., 0.]), 10000., m.pi/10 )

[0.         0.31415927 0.62831853 0.9424778  1.25663706 1.57079633
 1.88495559 2.19911486 2.51327412 2.82743339 3.14159265 3.45575192
 3.76991118 4.08407045 4.39822972 4.71238898 5.02654825 5.34070751
 5.65486678 5.96902604]
Current angle_index index is: 0
        ┌───────────┐
q526_0: ┤ U3(0,0,0) ├
        ├───────────┤
q526_1: ┤ U3(0,0,0) ├
        ├───────────┤
q526_2: ┤ U3(0,0,0) ├
        ├───────────┤
q526_3: ┤ U3(0,0,0) ├
        └───────────┘
c526_0: ═════════════
                     
c526_1: ═════════════
                     
The X contribution is: -78.125
The ZZ contribution is: -4.0
        ┌───────────────┐
q527_0: ┤ U3(pi/10,0,0) ├
        └─┬───────────┬─┘
q527_1: ──┤ U3(0,0,0) ├──
          ├───────────┤  
q527_2: ──┤ U3(0,0,0) ├──
          ├───────────┤  
q527_3: ──┤ U3(0,0,0) ├──
          └───────────┘  
c527_0: ═════════════════
                         
c527_1: ═════════════════
                         
The X contribution is: -4062.5
The ZZ contribution is: -3.8

The X contribution is: 3105.46875
The ZZ contribution is: -3.90234375
[   -82.125       -4066.38085938  -6546.59960938  -8167.22851562
  -9924.48242188 -10041.07421875  -9669.32226562  -8282.09375
  -6836.26953125  -2636.81640625   -957.03125      4296.76757812
   5370.7421875    7577.2265625    9354.16015625  10095.6171875
  10231.80273438   7242.8984375    5601.80273438   3101.56640625]
[1.57079633 0.         0.         0.        ]
Opt theta = 1.5707963267948966 for index: 0

Current angle_index index is: 1
        ┌──────────────┐
q546_0: ┤ U3(pi/2,0,0) ├
        └┬───────────┬─┘
q546_1: ─┤ U3(0,0,0) ├──
         ├───────────┤  
q546_2: ─┤ U3(0,0,0) ├──
         ├───────────┤  
q546_3: ─┤ U3(0,0,0) ├──
         └───────────┘  
c546_0: ════════════════
                        
c546_1: ════════════════
                        
The X contribution is: -10351.5625
The ZZ contribution is: -2.0234375
         ┌──────────────┐
q547_0: ─┤ U3(pi/2,0,0) ├
        ┌┴──────────────┤
q547_1: ┤ U3

The X contribution is: -7363.28125
The ZZ contribution is: -1.986328125
[-10353.5859375  -13185.47851562 -16056.48242188 -18282.87695312
 -19708.38671875 -19668.96289062 -19258.5546875  -18496.49023438
 -15449.43554688 -13554.68945312  -9765.61132812  -6484.43359375
  -4707.33984375  -1777.75195312    214.140625       96.5625
    467.41992188  -1915.65429688  -4415.87695312  -7365.26757812]
[1.57079633 1.25663706 0.         0.        ]
Opt theta = 1.2566370614359172 for index: 1

Current angle_index index is: 2
         ┌──────────────┐
q566_0: ─┤ U3(pi/2,0,0) ├
        ┌┴──────────────┤
q566_1: ┤ U3(2pi/5,0,0) ├
        └─┬───────────┬─┘
q566_2: ──┤ U3(0,0,0) ├──
          ├───────────┤  
q566_3: ──┤ U3(0,0,0) ├──
          └───────────┘  
c566_0: ═════════════════
                         
c566_1: ═════════════════
                         
The X contribution is: -20058.59375
The ZZ contribution is: -1.337890625
         ┌──────────────┐
q567_0: ─┤ U3(pi/2,0,0) ├
        ┌┴──────────

The X contribution is: -13339.84375
The ZZ contribution is: -1.048828125
         ┌──────────────┐
q585_0: ─┤ U3(pi/2,0,0) ├
        ┌┴──────────────┤
q585_1: ┤ U3(2pi/5,0,0) ├
        ├───────────────┤
q585_2: ┤ U3(5.969,0,0) ├
        └─┬───────────┬─┘
q585_3: ──┤ U3(0,0,0) ├──
          └───────────┘  
c585_0: ═════════════════
                         
c585_1: ═════════════════
                         
The X contribution is: -16250.0
The ZZ contribution is: -1.1640625
[-20059.93164062 -21446.59960938 -25196.41015625 -27910.8125
 -29140.99414062 -29609.36523438 -29042.67773438 -27616.43945312
 -25819.19726562 -22108.13476562 -19510.43945312 -16658.90039062
 -13827.1328125  -11932.87890625 -10194.86132812 -10156.26757812
  -9199.60546875 -11621.9140625  -13340.89257812 -16251.1640625 ]
[1.57079633 1.25663706 1.57079633 0.        ]
Opt theta = 1.5707963267948966 for index: 2

Current angle_index index is: 3
         ┌──────────────┐
q586_0: ─┤ U3(pi/2,0,0) ├
        ┌┴──────────────┤

The X contribution is: -21484.375
The ZZ contribution is: -0.03125
         ┌──────────────┐ 
q604_0: ─┤ U3(pi/2,0,0) ├─
        ┌┴──────────────┤ 
q604_1: ┤ U3(2pi/5,0,0) ├─
        └┬──────────────┤ 
q604_2: ─┤ U3(pi/2,0,0) ├─
        ┌┴──────────────┴┐
q604_3: ┤ U3(5.6549,0,0) ├
        └────────────────┘
c604_0: ══════════════════
                          
c604_1: ══════════════════
                          
The X contribution is: -23750.0
The ZZ contribution is: -0.01953125
         ┌──────────────┐
q605_0: ─┤ U3(pi/2,0,0) ├
        ┌┴──────────────┤
q605_1: ┤ U3(2pi/5,0,0) ├
        └┬──────────────┤
q605_2: ─┤ U3(pi/2,0,0) ├
        ┌┴──────────────┤
q605_3: ┤ U3(5.969,0,0) ├
        └───────────────┘
c605_0: ═════════════════
                         
c605_1: ═════════════════
                         
The X contribution is: -26406.25
The ZZ contribution is: -0.013671875
[-29355.4921875  -32382.78515625 -34999.95117188 -37558.63867188
 -38867.13085938 -39570.39257812 -39043.0

array([1.57079633, 1.25663706, 1.57079633, 1.57079633])

In [22]:
theta1 = 0
theta2 = 0
theta3 = 0
theta4 = 0

params = np.array([theta1, theta2, theta3, theta4])
h = 0.0

q = QuantumRegister(4)
c = ClassicalRegister(2) # we only need two classical bits since we at most take 2 measurements
qc = QuantumCircuit(q, c)

print(expectation(params, h))


entangled_ansatz(np.array([0,0,0,0,0]), qc)

        ┌───────────┐
q150_0: ┤ U3(0,0,0) ├
        ├───────────┤
q150_1: ┤ U3(0,0,0) ├
        ├───────────┤
q150_2: ┤ U3(0,0,0) ├
        ├───────────┤
q150_3: ┤ U3(0,0,0) ├
        └───────────┘
c150_0: ═════════════
                     
c150_1: ═════════════
                     
The X contribution is: 0.0
The ZZ contribution is: -4.0
-4.0


## References

https://arxiv.org/pdf/1804.03719.pdf